## VectorStore 지원 검색기

In [1]:
from langchain_community.document_loaders import TextLoader

# TextLoader를 사용하여 "./data/appendix-keywords.txt" 파일을 로드합니다.
loader = TextLoader("./data/appendix-keywords.txt")

- loader를 사용하여 문서를 로드합니다.
- CharacterTextSplitter를 사용하여 로드된 문서를 분할하고, 분할된 텍스트를 texts 변수에 저장합니다.
- OpenAIEmbeddings를 사용하여 임베딩 객체를 생성합니다.
- FAISS.from_documents 메서드를 사용하여 texts와 embeddings로부터 FAISS 벡터 데이터베이스 db를 생성합니다.

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# 문서를 로드합니다.
documents = loader.load()
# 문자 기반으로 텍스트를 분할하는 CharacterTextSplitter를 생성합니다. 청크 크기는 1000이고 청크 간 중복은 없습니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
# 로드된 문서를 분할합니다.
texts = text_splitter.split_documents(documents)
# OpenAI 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()
# 분할된 텍스트와 임베딩을 사용하여 FAISS 벡터 데이터베이스를 생성합니다.
db = FAISS.from_documents(texts, embeddings)

In [5]:
# 데이터베이스를 검색기로 사용하기 위해 retriever 변수에 할당합니다.
retriever = db.as_retriever()

In [6]:
# 관련 문서를 검색합니다.
docs = retriever.get_relevant_documents("임베딩(Embedding)은 무엇인가요?")
docs[0]

c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Document(metadata={'source': './data/appendix-keywords.txt'}, page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken')

### Max Marginal Relevance (MMR)

In [7]:
# 데이터베이스를 검색기로 사용하며, MMR(Maximal Marginal Relevance) 검색 유형을 지정합니다.
retriever = db.as_retriever(search_type="mmr")

In [8]:
# 관련 문서를 검색합니다.
docs = retriever.get_relevant_documents("임베딩(Embedding)은 무엇인가요?")
docs[0]

Document(metadata={'source': './data/appendix-keywords.txt'}, page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken')

### 유사도 점수 임계값 검색기(Similarity Score Threshold Retrieval)

In [9]:
retriever = db.as_retriever(
    # 검색 유형을 "유사도 점수 임계값"으로 설정합니다.
    search_type="similarity_score_threshold",
    # 검색 인자로 점수 임계값을 0.5로 지정합니다.
    search_kwargs={"score_threshold": 0.75},
)

In [10]:
# 관련 문서를 검색합니다.
docs = retriever.get_relevant_documents("임베딩(Embedding)은 무엇인가요?")
# 전체 검색 결과를 출력합니다.
docs

[Document(metadata={'source': './data/appendix-keywords.txt'}, page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken'),
 Document(metadata={'source': './data/appendix-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)'),
 Document(metadata={'source': './data/appendix-keywords.txt'}, page_content='Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding')]

### Top K 명시

In [11]:
# 데이터베이스를 검색하여 가장 관련성이 높은 1개의 결과를 반환하는 retriever를 생성합니다.
retriever = db.as_retriever(search_kwargs={"k": 1})

In [12]:
# 관련 문서를 검색합니다.
docs = retriever.get_relevant_documents("임베딩(Embedding)은 무엇인가요?")
# 전체 검색 결과를 출력합니다.
print(docs)
# 문서의 개수를 출력합니다.
print(f"문서의 개수: {len(docs)}")

[Document(metadata={'source': './data/appendix-keywords.txt'}, page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken')]
문서의 개수: 1
